# FinTech Demo

- Showcases how LLM pipelines can be done easily
- Created 2 Nov 2024 by John Tan Chong Min
- Modified 3 Dec for better citations

It has:
- strict_json to extract information from webpage
- citation from pdf with sources cited
- multi-step agentic retrieval augmented generation (RAG)

In [1]:
from agentjo import *

In [2]:
import os
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

In [4]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [5]:
llm('You are a friendly assistant', 'Give me 5 names')

'Sure! Here are five names:\n\n1. Olivia\n2. Ethan\n3. Sophia\n4. Liam\n5. Ava\n\nLet me know if you need more names or specific types!'

# StrictJSON
- Website: https://www.straitstimes.com/business/mas-sets-up-new-not-for-profit-firm-as-it-seeks-to-extend-reach-to-global-fintech-sector

In [6]:
text = '''SINGAPORE – The Monetary Authority of Singapore (MAS) has set up a new not-for-profit firm focusing on fintech as it moves to expand its network globally.

The new company, called the Global Finance & Technology Network (GFTN), has four business lines – to organise fintech conferences, provide advisory and research services, provide digital platform services to small and medium-sized enterprises, and grow an investment fund for technology start-ups.

MAS’ not-for-profit entity Elevandi, which was set up in 2021 to advance fintech in the global digital economy, will be folded into the new firm.

“Through the connectivity to Singapore because we are Singapore-based... all our connections will come to Singapore. That benefits the Singapore fintech ecosystem,” said Mr Menon, who is now Singapore’s Ambassador for Climate Action and senior adviser to the National Climate Change Secretariat in the Prime Minister’s Office.

He said that GFTN will strive to be self-sustaining, like Elevandi was, adding that he expects GFTN to break even in three to five years’ time. But before GFTN can sustain itself, MAS will provide seed funding of an undisclosed amount.

The first order of business for the team is for the board to spend the next few months refining strategies and setting goals for each year in the next five years.

As for its investment fund business, it will be a subsidiary of the new company.

The funds will come from limited partners or third-party sources that GFTN attracts around the world. This subsidiary will apply for a licence from MAS.

GFTN’s advisory business will focus on emerging markets from Africa to South-east Asia.

Mr Menon said he is chairing GFTN because his “interest in technology continues”.

“It’s simply because of what I used to do in MAS and the connections I had there and so on,” he said.

“Both (sustainability and innovation) are actually issues that are very close to my heart... the planet and people,” he added.

And as a retiree, he has “a little bit of bandwidth, so I can also do this”.

Both MAS deputy managing director Leong Sing Chiong and Nominated MP Neil Parekh, who is also non-executive chairman of Tikehau Capital, have been appointed GFTN’s deputy chairmen. 

GFTN’s group chief executive will be MAS’ current chief fintech officer Sopnendu Mohanty, who will relinquish his MAS role from Feb 1, 2025.

He will continue to advise on fintech matters at MAS.

Mr Kenneth Gay, currently an executive director heading MAS’ enterprise knowledge department, will take over as the new head of fintech at the authority on Feb 1.'''

In [7]:
res = strict_json("Summarise and give the sentiment for the text",
                  text,
                  output_format = {"Summary": "Summarise in three words",
                                   "Sentiment": "type: Enum['Positive', 'Negative', 'Neutral']",
                                   "People involved": "list in alphabetic order, type: list",
                                   "Names of corporations": "list in alphabetic order, type: list"},
                  llm = llm)

In [8]:
res

{'Summary': 'Fintech network established',
 'Sentiment': 'Positive',
 'People involved': ['Kenneth Gay',
  'Leong Sing Chiong',
  'Menon',
  'Neil Parekh',
  'Sopnendu Mohanty'],
 'Names of corporations': ['Global Finance & Technology Network',
  'Monetary Authority of Singapore',
  'Tikehau Capital']}

In [9]:
res['People involved']

['Kenneth Gay',
 'Leong Sing Chiong',
 'Menon',
 'Neil Parekh',
 'Sopnendu Mohanty']

# Memory from File
- Download pdf from https://www.mckinsey.com/industries/financial-services/our-insights/global-payments-in-2024-simpler-interfaces-complex-reality?cid=soc-web#/

In [10]:
memory = Memory()
memory.add_file('Global payments in 2024 - Mc Kinsey report.pdf')

In [11]:
memory.top_k = 3
memory.memory[:5]

['The 2024 McKinsey Global Payments Report\nGlobal payments in 2024: \nSimpler interfaces,  \ncomplex reality \nOctober 2024\nThe 2024 McKinsey Global Payments Report\nGlobal payments in 2024: \nSimpler interfaces,  \ncomplex reality\nBy Philip Bruno and Uzayr Jeenah,\nwith Amit Gandhi and Inês Gancho\nThe tantalizing vision for global payments has long been that they’d be safe, simple, quick, inexpensive, and',
 'ubiquitous. This promise has come closer with each new technological advance. However, instant payment \nsystems have ushered in the Decoupled Era, a term coined in last year’s Global Payments Report.1 In this new \nworld, while payments are getting quicker, they are increasingly disconnected from accounts, and players \nhave proliferated, fragmenting the value chain, increasing complexity, and endangering the vision',
 '.\nWith $1.8 quadrillion flows in global payments annually, even small shifts in how payments are delivered \ncan reshape not just companies but entire secto

## Write an answer according to citation

In [12]:
question = 'How many transactions did the global payments industry handle in 2023?'
mem = memory.retrieve(question)

In [13]:
# add in citation numbers
for i in range(len(mem)):
    mem[i] = f'[{i+1}] '+ mem[i]

In [14]:
mem

['[1] .\nGlobal payments revenues will continue to grow\nIn 2023, the global payments industry handled 3.4 trillion transactions, accounting for $1.8 quadrillion in \nvalue and a revenue pool of $2.4 trillion. Revenues grew 7 percent annually from 2018 to 2023, supported by \ncontinued consumer and commercial transaction digitization and higher interest rates. Our analysis suggests \nrevenue growth will be slower in the next few years at 5 percent a year, in line with expected nominal GDP',
 '[2] .\n3  “Global Payments Map,” McKinsey, accessed October 2024.In 2023, the global payments industry handled  \n3.4 trillion transactions  \nGlobal payments in 2024: Simpler interfaces, complex reality 2\nNet interest income contributes approximately 47 percent to the total payments revenue pool. This represents \na three-percentage-point share increase from 2021, a year marked by historically low interest rates in most \nmarkets',
 '[3] 3 Global payments in 2024: Simpler interfaces, complex rea

In [15]:
res = strict_json(f'''Write an answer to the question and cite the sources according to number. 
Answer only according to the references, output no info if references are not relevant
Use the references [1], [2], [3]: 
{question}''',
                  mem,
                  output_format = {'Answer': 'Answer with the references, type: str'},
                  llm = llm)

In [16]:
res

{'Answer': 'In 2023, the global payments industry handled 3.4 trillion transactions [1][2].'}

# Agent with Memory function

In [17]:
def get_knowledge(query: str) -> str:
    ''' Retrieves relevant information according to query '''
    mem = memory.retrieve(query)
    # add in citation numbers
    for i in range(len(mem)):
        mem[i] = f'[{i+1}] '+ mem[i]
    res = strict_json(f'''Write an answer to the question and cite the sources according to number. 
Answer only according to the references, output no info if references are not relevant
Use the references [1], [2], [3]: 
{query}?''',
              mem,
              output_format = {'Answer': 'Answer with the references, type: str'},
              llm = llm)
    return 'Answer: '+ res['Answer'] + '\n\nSources:\n' + str(mem)

In [18]:
agent = Agent('Question Answerer', 
              'Answers questions factually based on retrieved knowledge. You should split the query into multiple parts and query separately',
              default_to_llm = False,
              llm = llm).assign_functions([get_knowledge])

In [19]:
agent.print_functions()

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: get_knowledge
Description:  Retrieves relevant information according to <query: str> 
Input: ['query']
Output: {'output_1': 'str'}



In [54]:
agent.reset()
output = agent.run('What is the number of transactions the global payments industry handled in 2023?')

Observation: No subtasks have been completed yet for the assigned task regarding the number of transactions in the global payments industry for 2023.
Thoughts: To complete the assigned task, I need to retrieve relevant information about the number of transactions handled by the global payments industry in 2023. This can be done by querying the appropriate data source.
Subtask identified: Query the knowledge base for the number of transactions the global payments industry handled in 2023.
Calling function get_knowledge with parameters {'query': 'number of transactions the global payments industry handled in 2023'}
> {'output_1': "Answer: In 2023, the global payments industry handled 3.4 trillion transactions [1][2].\n\nSources:\n['[1] .\\nGlobal payments revenues will continue to grow\\nIn 2023, the global payments industry handled 3.4 trillion transactions, accounting for $1.8 quadrillion in \\nvalue and a revenue pool of $2.4 trillion. Revenues grew 7 percent annually from 2018 to 202

In [55]:
output = agent.reply_user('''Reply in the following format: 
Answer: <answer with in-linne citation of relevant sources with [x], where x is a number>
Sources: <only display relevant sources stated in answer separated by newline. Display the full source>''')

Answer: In 2023, the global payments industry handled 3.4 trillion transactions [1][2].

Sources:
[1] .\
Global payments revenues will continue to grow\
In 2023, the global payments industry handled 3.4 trillion transactions, accounting for $1.8 quadrillion in \
value and a revenue pool of $2.4 trillion. Revenues grew 7 percent annually from 2018 to 2023, supported by \
continued consumer and commercial transaction digitization and higher interest rates. Our analysis suggests \
revenue growth will be slower in the next few years at 5 percent a year, in line with expected nominal GDP
[2] .\
3  “Global Payments Map,” McKinsey, accessed October 2024.In 2023, the global payments industry handled  \
3.4 trillion transactions  \
Global payments in 2024: Simpler interfaces, complex reality 2\
Net interest income contributes approximately 47 percent to the total payments revenue pool. This represents \
a three-percentage-point share increase from 2021, a year marked by historically low interes

In [56]:
agent.status()

Agent Name: Question Answerer
Agent Description: Answers questions factually based on retrieved knowledge. You should split the query into multiple parts and query separately
Available Functions: ['end_task', 'get_knowledge']
Shared Variables: ['agent']
Task: What is the number of transactions the global payments industry handled in 2023?
Subtasks Completed:
Subtask: get_knowledge(query="number of transactions the global payments industry handled in 2023")
{'output_1': "Answer: In 2023, the global payments industry handled 3.4 trillion transactions [1][2].\n\nSources:\n['[1] .\\nGlobal payments revenues will continue to grow\\nIn 2023, the global payments industry handled 3.4 trillion transactions, accounting for $1.8 quadrillion in \\nvalue and a revenue pool of $2.4 trillion. Revenues grew 7 percent annually from 2018 to 2023, supported by \\ncontinued consumer and commercial transaction digitization and higher interest rates. Our analysis suggests \\nrevenue growth will be slower in

In [57]:
agent.thoughts

[{'Observation': 'No subtasks have been completed yet for the assigned task regarding the number of transactions in the global payments industry for 2023.',
  'Thoughts': 'To complete the assigned task, I need to retrieve relevant information about the number of transactions handled by the global payments industry in 2023. This can be done by querying the appropriate data source.',
  'Current Subtask': 'Query the knowledge base for the number of transactions the global payments industry handled in 2023.',
  'Equipped Function Name': 'get_knowledge',
  'Equipped Function Inputs': {'query': 'number of transactions the global payments industry handled in 2023'}},
 {'Observation': 'The query regarding the number of transactions handled by the global payments industry in 2023 has been successfully answered, revealing that there were 3.4 trillion transactions.',
  'Thoughts': 'Since the main question has been answered, the next step is to summarize the findings and present the final output t

In [58]:
agent.subtasks_completed

{'get_knowledge(query="number of transactions the global payments industry handled in 2023")': {'output_1': "Answer: In 2023, the global payments industry handled 3.4 trillion transactions [1][2].\n\nSources:\n['[1] .\\nGlobal payments revenues will continue to grow\\nIn 2023, the global payments industry handled 3.4 trillion transactions, accounting for $1.8 quadrillion in \\nvalue and a revenue pool of $2.4 trillion. Revenues grew 7 percent annually from 2018 to 2023, supported by \\ncontinued consumer and commercial transaction digitization and higher interest rates. Our analysis suggests \\nrevenue growth will be slower in the next few years at 5 percent a year, in line with expected nominal GDP', '[2] .\\n3  “Global Payments Map,” McKinsey, accessed October 2024.In 2023, the global payments industry handled  \\n3.4 trillion transactions  \\nGlobal payments in 2024: Simpler interfaces, complex reality 2\\nNet interest income contributes approximately 47 percent to the total payment

# Multi-step retrieval

In [36]:
agent.reset()
output = agent.run('How much will growth be, and what are the 3 main areas to capture them?')

Observation: No subtasks have been completed yet, and the Assigned Task requires information on growth and its main areas.
Thoughts: To complete the Assigned Task, I need to break down the query into two parts: one for understanding the growth amount and another for identifying the three main areas related to growth.
Subtask identified: Query for the amount of growth.
Calling function get_knowledge with parameters {'query': 'What is the projected amount of growth in the market for the next year?'}
> {'output_1': "Answer: The projected amount of growth in the market for the next year is expected to be 5 percent, continuing from the overall trend of revenue growth in the payments industry [1][2].\n\nSources:\n['[1] .\\nWe start by examining the overall payments industry, how it has grown, and the rate at which that growth can \\nbe expected to continue. Then, we look at six trends shaping the industry over the next several years. Lastly, \\nwe describe three areas where operators and dis

In [38]:
output = agent.reply_user('Reply in the form of an executive summary')

Executive Summary: The payments industry is projected to experience a growth rate of 5 percent over the next year, continuing the trend of revenue growth observed in recent years. This growth is expected to be driven by ongoing consumer and commercial transaction digitization, as well as higher interest rates. By 2028, total payments revenues are anticipated to reach approximately $3.1 trillion, which represents a significant portion of the total banking revenue pool.

To capitalize on this growth, operators and disrupters in the payments industry should focus their investments in three key areas: 1) instant payments offerings in margin-rich segments, which present opportunities for success; 2) enhancements to anti-money laundering processes to ensure compliance and security; and 3) the development of robust, real-time infrastructure and technology aimed at improving payment efficiency and fraud prevention. These strategic investments are essential for staying competitive in an evolvin